# Libs

In [1]:
import warnings
warnings.filterwarnings('ignore')

import gurobipy as grb
from gurobipy import *
import numpy as np
import geatpy as ea

import torch
from torch.autograd import Variable

# Build Models

In [ ]:
class SPP_gurobi():
    def __init__(self, num_intervals, num_pairs, num_links, impact_intervals):
        self.num_intervals = num_intervals
        self.num_pairs = num_pairs
        self.num_links = num_links
        self.imp_intervals = impact_intervals
    
    
    def build_model(self, prior_od, prior_link, assignment_fraction, mcr):
        
        assert prior_od.shape == (self.num_intervals, self.num_pairs)
        assert prior_link.shape == (self.num_intervals, self.num_links)
        assert assignment_fraction.shape == (self.num_pairs, self.num_links, self.imp_intervals)
        
        # create model
        spp = Model("SPP")
        
        # create varibles
        X = spp.addVars(['X_' + str(i) for i in range(self.num_intervals * self.num_pairs)],
                        lb=0, obj=1, vtype=GRB.CONTINUOUS)
        Y = spp.addVars(['Y_' + str(i) for i in range(self.num_intervals * self.num_links)],
                        lb=0, obj=1, vtype=GRB.CONTINUOUS)
        X = np.array(X.values()).reshape(self.num_intervals, self.num_pairs)
        Y = np.array(Y.values()).reshape(self.num_intervals, self.num_links)
        spp.update()
        
        # build objective
        X_hat = prior_od
        Y_hat = prior_link
        X_var = np.var(prior_od)
        Y_var = np.var(prior_link)
        
        obj = 0
        for k in range(self.num_intervals):
            for i in range(self.num_pairs):
                obj += (X[k,i] - X_hat[k,i]) * (X[k,i] - X_hat[k,i]) / X_var
        
        for k in range(self.num_intervals):
            for j in range(self.num_links):
                obj += (Y[k,j] - Y_hat[k,j]) * (Y[k,j] - Y_hat[k,j]) / Y_var
        
        spp.setObjective(obj, GRB.MINIMIZE)
        
        # add constraints
        # No.1: Linear Assignment
        T = self.imp_intervals
        A = assignment_fraction
        for k in range(self.num_intervals):
            t = 0
            rhs = 0
            while (k - t >= 0) & (t <= T):
                rhs += (np.dot(X[k - t,:], A[:,:,t - 1])).reshape(1,self.num_links)
                t += 1
            for j in range(self.num_links):
                spp.addConstr(Y[k,j] == rhs[0,j])
        
        # No.2: Maximum Change Ratio
        beta = mcr
        for k in range(1,self.num_intervals):
            for i in range(self.num_pairs):
                spp.addConstr(X[k,i] >= (1 - beta) * X[k - 1,i])
                spp.addConstr(X[k,i] <= (1 + beta) * X[k - 1,i])
        
        return spp

    
    def estimate(self, prior_od, prior_link, assignment_fraction, mcr):
        
        spp = self.build_model(prior_od, prior_link, assignment_fraction, mcr)
        spp.optimize()
        
        x_range = self.num_intervals * self.num_pairs
        solution_x = [v.x for v in spp.getVars()[:x_range]]
        solution_y = [v.x for v in spp.getVars()[x_range:]]
        
        return solution_x, solution_y

In [ ]:
class PRA_gurobi():
    def __init__(self, num_intervals, num_pairs, num_links, impact_intervals):
        self.num_intervals = num_intervals
        self.num_pairs = num_pairs
        self.num_links = num_links
        self.imp_intervals = impact_intervals
    
    
    def build_model(self, prior_od, prior_link, prior_pr, probe_od, assignment_fraction, assignment_fraction_pr, mcr):
        
        assert prior_od.shape == (self.num_intervals, self.num_pairs)
        assert prior_link.shape == (self.num_intervals, self.num_links)
        assert prior_pr.shape == (self.num_intervals, self.num_links)
        assert probe_od.shape == (self.num_intervals, self.num_pairs)
        assert assignment_fraction.shape == (self.num_pairs, self.num_links, self.imp_intervals)
        assert assignment_fraction_pr.shape == (self.num_pairs, self.num_links, self.imp_intervals)
        
        # create model
        pra = Model("PRA")
        
        # create varibles
        X = pra.addVars(['X_' + str(i) for i in range(self.num_intervals * self.num_pairs)],
                        lb=0, obj=1, vtype=GRB.CONTINUOUS)
        Y = pra.addVars(['Y_' + str(i) for i in range(self.num_intervals * self.num_links)],
                        lb=0, obj=1, vtype=GRB.CONTINUOUS)
        Theta = pra.addVars(['T_' + str(i) for i in range(self.num_intervals * self.num_links)],
                        lb=0, ub=1, obj=1, vtype=GRB.CONTINUOUS)
        H = pra.addVars(['H_' + str(i) for i in range(self.num_intervals * self.num_pairs)],
                        lb=0, ub=1, obj=1, vtype=GRB.CONTINUOUS)
        X = np.array(X.values()).reshape(self.num_intervals, self.num_pairs)
        Y = np.array(Y.values()).reshape(self.num_intervals, self.num_links)
        Theta = np.array(Theta.values()).reshape(self.num_intervals, self.num_links)
        H = np.array(H.values()).reshape(self.num_intervals, self.num_pairs)
        pra.update()
        
        # build objective
        X_hat = prior_od
        Y_hat = prior_link
        Theta_hat = prior_pr # OD penetration rates
        X_var = np.var(X_hat)
        Y_var = np.var(Y_hat)
        Theta_var = np.var(Theta_hat)
        
        obj = 0
        for k in range(self.num_intervals):
            for i in range(self.num_pairs):
                obj += (X[k,i] - X_hat[k,i]) / X_var
        
        for k in range(self.num_intervals):
            for j in range(self.num_links):
                obj += (Y[k,j] - Y_hat[k,j]) / Y_var
                obj += (Theta[k,j] - Theta_hat[k,j]) / Theta_var
        
        pra.setObjective(obj, GRB.MINIMIZE)
        
        # add constraints
        # No.1: Linear Assignment
        T = self.imp_intervals
        A = assignment_fraction
        for k in range(self.num_intervals):
            t = 0
            rhs = 0
            while (k - t >= 0) & (t <= T):
                rhs += (np.dot(X[k - t,:], A[:,:,t - 1])).reshape(1,self.num_links)
                t += 1
            for j in range(self.num_links):
                pra.addConstr(0.9 * Y[k,j] <= rhs[0,j])
                pra.addConstr(1.9 * Y[k,j] >= rhs[0,j])
        
        # No.2: Penetration Rate Assignment
        Z = probe_od
        Rho = assignment_fraction_pr
        for k in range(self.num_intervals):
            for i in range(self.num_pairs):
                pra.addConstr(X[k,i] * H[k,i] == Z[k,i])
        
        for k in range(self.num_intervals):
            t = 0
            rhs = 0
            while (k - t >= 0) & (t <= T):
                rhs += (np.dot(H[k - t,:], Rho[:,:,t - 1])).reshape(1,self.num_links)
                t += 1
            for j in range(self.num_links):
                pra.addConstr(0.9 * Theta[k,j] <= rhs[0,j])
                pra.addConstr(1.1 * Theta[k,j] >= rhs[0,j])
        
        # No.3: Maximum Change Ratio
        beta = mcr
        for k in range(1,self.num_intervals):
            for i in range(self.num_pairs):
                pra.addConstr(X[k,i] >= (1 - beta) * X[k - 1,i])
                pra.addConstr(X[k,i] <= (1 + beta) * X[k - 1,i])
        
        return pra

    
    def estimate(self, prior_od, prior_link, prior_pr, probe_od, assignment_fraction, assignment_fraction_pr, mcr):
        
        pra = self.build_model(prior_od, prior_link, prior_pr, probe_od, assignment_fraction, assignment_fraction_pr, mcr)
        pra.params.NonConvex = 2
        pra.params.Heuristics = 0.1
        pra.params.timeLimit = 180
        pra.Params.MIPFocus = 1
        pra.optimize()
        
        x_range = self.num_intervals * self.num_pairs
        y_range = self.num_intervals * (self.num_pairs + self.num_links)
        t_range = self.num_intervals * (self.num_pairs + 2 * self.num_links)
        solution_x = [v.x for v in pra.getVars()[:x_range]]
        solution_y = [v.x for v in pra.getVars()[x_range:y_range]]
        solution_t = [v.x for v in pra.getVars()[y_range:t_range]]
        solution_h = [v.x for v in pra.getVars()[t_range:]]
        
        return solution_x, solution_y, solution_t, solution_h

# Build Models and EA Solution

In [ ]:
class PRA_GA_SOEA(ea.Problem):
    def __init__(self, num_intervals, num_pairs, num_links,
                 prior_od, prior_link, prior_pr,
                 probe_od, assignment_fraction,
                 assignment_fraction_pr, M = 1):
        
        name = 'PRA'
        
        # number of decision variables
        Dim = num_intervals * num_pairs
        
        # objective type 1-min, -1-max
        maxormins = [1] * M
        
        # variable type 0-real, 1-integer
        varTypes = [0] * Dim
        
        lb = [0] * Dim
        ub = [500] * Dim
        
        # include bound
        lbin = [1] * Dim
        ubin = [1] * Dim
        
        # instantialize
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)
    
    def aimFunc(self, pop):
        
        # decision variables in the shape of (Dim,1)
        Vars = pop.Phen
        NIND = Vars.shape[0]
        
        def loss(X):
            # calculate objective
            X = X.reshape(num_intervals, num_pairs)

            # objective No.1
            f1 = np.sum((X - prior_od) ** 2)
            f1 /= np.var(prior_od)

            # objective No.2
            f2 = 0
            T = impact_intervals
            A = assignment_fraction
            for k in range(num_intervals):
                t = 0
                rhs = 0
                while (k - t >= 0) & (t <= T):
                    rhs += (np.dot(X[k - t,:], A[:,:,t - 1])).reshape(1,num_links)
                    t += 1
                f2 += np.sum((rhs - prior_link[k,:]) ** 2)
            f2 /= np.var(prior_link)

            # objective No.3
            f3 = 0
            H = prior_od / (X + 1e-3)
            Rho = assignment_fraction_pr
            for k in range(num_intervals):
                t = 0
                rhs = 0
                while (k - t >= 0) & (t <= T):
                    rhs += (np.dot(H[k - t,:], Rho[:,:,t - 1])).reshape(1,num_links)
                    t += 1
                f3 += np.sum((rhs - prior_pr[k,:]) ** 2)
            f3 /= np.var(prior_pr)

            return f1 + f2 + f3
        
        loss_NIND = [loss(Vars[i]) for i in range(NIND)]
        pop.ObjV = np.array(loss_NIND).reshape(NIND,1)

        
class SPP_GA_SOEA(ea.Problem):
    def __init__(self, num_intervals, num_pairs, num_links,
                 prior_od, prior_link, assignment_fraction, M = 1):
        
        name = 'SPP'
        
        # number of decision variables
        Dim = num_intervals * num_pairs
        
        # objective type 1-min, -1-max
        maxormins = [1] * M
        
        # variable type 0-real, 1-integer
        varTypes = [0] * Dim
        
        lb = [0] * Dim
        ub = [500] * Dim
        
        # include bound
        lbin = [1] * Dim
        ubin = [1] * Dim
        
        # instantialize
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)
    
    def aimFunc(self, pop):
        
        # decision variables in the shape of (Dim,1)
        Vars = pop.Phen
        NIND = Vars.shape[0]
        
        def loss(X):
            # calculate objective
            X = X.reshape(num_intervals, num_pairs)

            # objective No.1
            f1 = np.sum((X - prior_od) ** 2)
            f1 /= np.var(prior_od)

            # objective No.2
            f2 = 0
            T = impact_intervals
            A = assignment_fraction
            for k in range(num_intervals):
                t = 0
                rhs = 0
                while (k - t >= 0) & (t <= T):
                    rhs += (np.dot(X[k - t,:], A[:,:,t - 1])).reshape(1,num_links)
                    t += 1
                f2 += np.sum((rhs - prior_link[k,:]) ** 2)
            f2 /= np.var(prior_link)

            return f1 + f2
        
        loss_NIND = [loss(Vars[i]) for i in range(NIND)]
        pop.ObjV = np.array(loss_NIND).reshape(NIND,1)

        
class PRA_GA_MOEA(ea.Problem):
    def __init__(self, num_intervals, num_pairs, num_links,
                 prior_od, prior_link, prior_pr,
                 probe_od, assignment_fraction,
                 assignment_fraction_pr, M = 3):
        
        name = 'PRA'
        
        # number of decision variables
        Dim = num_intervals * num_pairs
        
        # objective type 1-min, -1-max
        maxormins = [1] * M
        
        # variable type 0-real, 1-integer
        varTypes = [0] * Dim
        
        lb = [0] * Dim
        ub = [500] * Dim
        
        # include bound
        lbin = [1] * Dim
        ubin = [1] * Dim
        
        # instantialize
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)
    
    def aimFunc(self, pop):
        
        # decision variables in the shape of (Dim,1)
        Vars = pop.Phen
        NIND = Vars.shape[0]
        
        def loss(X):
            # calculate objective
            X = X.reshape(num_intervals, num_pairs)

            # objective No.1
            f1 = np.sum((X - prior_od) ** 2)
            f1 /= np.var(prior_od)

            # objective No.2
            f2 = 0
            T = impact_intervals
            A = assignment_fraction
            for k in range(num_intervals):
                t = 0
                rhs = 0
                while (k - t >= 0) & (t <= T):
                    rhs += (np.dot(X[k - t,:], A[:,:,t - 1])).reshape(1,num_links)
                    t += 1
                f2 += np.sum((rhs - prior_link[k,:]) ** 2)
            f2 /= np.var(prior_link)

            # objective No.3
            f3 = 0
            H = prior_od / (X + 1e-3)
            Rho = assignment_fraction_pr
            for k in range(num_intervals):
                t = 0
                rhs = 0
                while (k - t >= 0) & (t <= T):
                    rhs += (np.dot(H[k - t,:], Rho[:,:,t - 1])).reshape(1,num_links)
                    t += 1
                f3 += np.sum((rhs - prior_pr[k,:]) ** 2)
            f3 /= np.var(prior_pr)
            
            f1 = np.array(f1).reshape(-1,1)
            f2 = np.array(f2).reshape(-1,1)
            f3 = np.array(f3).reshape(-1,1)
            return np.hstack([f1, f2, f3])
        
        loss_NIND = [loss(Vars[i]) for i in range(NIND)]
        pop.ObjV = np.array(loss_NIND).reshape(NIND,3)

        
class SPP_GA_MOEA(ea.Problem):
    def __init__(self, num_intervals, num_pairs, num_links,
                 prior_od, prior_link, assignment_fraction, M = 2):
        
        name = 'SPP'
        
        # number of decision variables
        Dim = num_intervals * num_pairs
        
        # objective type 1-min, -1-max
        maxormins = [1] * M
        
        # variable type 0-real, 1-integer
        varTypes = [0] * Dim
        
        lb = [0] * Dim
        ub = [500] * Dim
        
        # include bound
        lbin = [1] * Dim
        ubin = [1] * Dim
        
        # instantialize
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)
    
    def aimFunc(self, pop):
        
        # decision variables in the shape of (Dim,1)
        Vars = pop.Phen
        NIND = Vars.shape[0]
        
        def loss(X):
            # calculate objective
            X = X.reshape(num_intervals, num_pairs)

            # objective No.1
            f1 = np.sum((X - prior_od) ** 2)
            f1 /= np.var(prior_od)

            # objective No.2
            f2 = 0
            T = impact_intervals
            A = assignment_fraction
            for k in range(num_intervals):
                t = 0
                rhs = 0
                while (k - t >= 0) & (t <= T):
                    rhs += (np.dot(X[k - t,:], A[:,:,t - 1])).reshape(1,num_links)
                    t += 1
                f2 += np.sum((rhs - prior_link[k,:]) ** 2)
            f2 /= np.var(prior_link)
            
            f1 = np.array(f1).reshape(-1,1)
            f2 = np.array(f2).reshape(-1,1)
            
            return np.hstack([f1, f2])
        
        loss_NIND = [loss(Vars[i]) for i in range(NIND)]
        pop.ObjV = np.array(loss_NIND).reshape(NIND,2)

In [ ]:
class solve_problem():
    def __init__(self, problem_type):
        assert problem_type in ['soea','moea']
        self.problem_type = problem_type
        if problem_type is 'soea':
            self.algorithm = ea.soea_SEGA_templet
        else:
            self.algorithm = ea.moea_NSGA2_templet
    
    def fit(self, problem, population_size, max_generation, encoding='RI'):
        
        Encoding = encoding
        NIND = population_size
        Field = ea.crtfld(Encoding, problem.varTypes,
                          problem.ranges, problem.borders)
        population = ea.Population(Encoding, Field, NIND)
        algo = self.algorithm(problem, population)
        algo.MAXGEN = max_generation
        
        if self.problem_type is 'soea':
            [population, obj_trace, var_trace] = algo.run()
            return population
        elif self.problem_type is 'moea':
            NDSet = algo.run()
            return NDSet

# Instantialize

In [ ]:
# params init
num_intervals = 18
num_pairs = 100
num_links = 4
impact_intervals = 3

# prior information
prior_od = 20 + 5 * np.random.rand(num_intervals, num_pairs)
prior_link = 10 + 10 * np.random.rand(num_intervals, num_links)
prior_pr = 0.1 + 0.02 * np.random.normal(0,1,(num_intervals, num_links))
probe_od = 1 + 1 * np.random.rand(num_intervals, num_pairs)
assignment_fraction = np.random.rand(num_pairs,num_links,impact_intervals)
assignment_fraction_pr = np.random.rand(num_pairs,num_links,impact_intervals)

In [ ]:
# optimization model
spp = SPP_gurobi(num_intervals, num_pairs, num_links, impact_intervals)
x, y = spp.estimate(prior_od, prior_link,assignment_fraction, 0.2)

In [ ]:
# optimization model
pra = PRA_gurobi(num_intervals, num_pairs, num_links, impact_intervals)
x, y = pra.estimate(prior_od, prior_link, prior_pr, probe_od,
                    assignment_fraction, assignment_fraction_pr, 0.2)

In [ ]:
# heuristic solution
problem = PRA_GA_MOEA(num_intervals, num_pairs, num_links,
                      prior_od, prior_link, prior_pr,
                      probe_od, assignment_fraction,
                      assignment_fraction_pr)
solver = solve_problem(problem_type='moea')
population = solver.fit(problem,population_size=5,max_generation=100)

# Torch autograd

In [ ]:
X = Variable(torch.rand((18,150)),requires_grad=True)
w = torch.rand((150,20))
X_hat = torch.rand((18,150))
y = torch.matmul(X,w).sum() + ((X - X_hat) ** 2).sum()

y.backward()
X.grad

# Bilinear model

In [2]:
m = Model('bilinear')
x = m.addVar(lb=0,obj=1,vtype=GRB.CONTINUOUS)
y = m.addVar(lb=0,obj=1,vtype=GRB.CONTINUOUS)
z = m.addVar(lb=0,obj=1,vtype=GRB.CONTINUOUS)
m.update()
m.setObjective(x * y + z,GRB.MINIMIZE)
m.addConstr(x * y == 10)
m.addConstr(z * x == 50)
m.addConstr(x + y + z <= 30)
m.params.NonConvex = 2
m.params.MIPFocus = 1
m.optimize()

Using license file C:\Users\Cao\Desktop\softwares\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x8310b124
Model has 1 quadratic objective term
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+01]
  QRHS range       [1e+01, 5e+01]

Continuous model is non-convex -- solving as a MIP.

Presolve time: 0.02s
Presolved: 11 rows, 4 columns, 25 nonzeros
Presolved model has 3 bilinear constraint(s)
Variable types: 4 continuous, 0 integer (0 binary)

Root relaxation: objective 5.151921e+00,

# Graph Partition

In [2]:
# number of groups (consider a 5x5 net)
group_num = 16

# number of nodes
node_num = 100

# variables
ajacency = np.round(np.random.rand(node_num, node_num))
weight = np.random.rand(node_num, node_num) * ajacency

# ======== DEFINE Model ========
m = Model("Graph_Partition")

# ======== DEFINE DECISION VARIABLE ========
X = m.addVars(['X_' + str(i) for i in range(group_num * node_num)],
                lb=0, obj=1, vtype=GRB.BINARY)
X = np.array(X.values()).reshape(group_num, node_num)
m.update()

Using license file C:\Users\Cao\Desktop\softwares\gurobi.lic
Academic license - for non-commercial use only


In [11]:
# ======== DEFINE OBJECTIVE ========
obj = 0
for group_idx in range(group_num):
    
    # calculate group & cut connection state
    group_vec = X[group_idx, :].reshape(-1, 1)
    cut_vec = 1 - X[group_idx, :].reshape(-1, 1)
    group_connection_state = ajacency * np.matmul(group_vec, group_vec.T)
    cut_connection_state = ajacency * np.matmul(cut_vec, cut_vec.T)
    
    # Obj Term 1: positive gain from high connectivity coefficient
    obj += np.sum(group_connection_state * weight)
    
    # Obj Term 2: elimination loss from cuts
    obj -= np.sum(cut_connection_state * weight)
m.setObjective(obj, GRB.MAXIMIZE)

# ======== DEFINE CONSTRAINTS========
for group_idx in range(group_num):
    
    # calculate group & cut connection state
    group_vec = X[group_idx, :].reshape(-1, 1)
    group_connection_state = ajacency * np.matmul(group_vec, group_vec.T)
    
    # Constr 1: nodes in same group must connect
    m.addConstr(np.sum(group_connection_state) == np.sum(group_vec))
    
    # Constr 2: each node should be contained in a group
    m.addConstr(np.sum(X, axis=1) == 1)
    
    # Constr 2: 0 < node number in each group < max_num
    m.addConstr(np.sum(group_vec) >= 1)
    m.addConstr(np.sum(group_vec) <= 10)

In [ ]:
m.params.NonConvex = 2
m.optimize()
solution_x = [v.x for v in m.getVars()]